In [1]:
import torch 
import numpy 
from sklearn.datasets import make_classification

In [2]:
dump_ = 1

def dump_info(tensor_,title):
    if dump_ == 1:
        print(title)
        print( list(tensor_.size() ))
        print(tensor_)
        
class Feedforward(torch.nn.Module):
        def __init__(self, input_size, hidden_size):
            print("Feedforward input_size ",input_size," hidden_size ", hidden_size)
            super(Feedforward, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size, 1)
            self.sigmoid = torch.nn.Sigmoid()        
        
        def forward(self, x):
            global dump_
            
            dump_info(x,"input")
            
            hidden = self.fc1(x)            
            dump_info(hidden,"hidden fc1")
            
            relu = self.relu(hidden)            
            dump_info(relu,"relu")
            
            output = self.fc2(relu)            
            dump_info(output,"output fc2")
            
            sigmoid_output = self.sigmoid(output)
            dump_info(sigmoid_output,"sigmoid_output")
            #close logging
            dump_ = 2
            
            return sigmoid_output

In [3]:
# CREATE RANDOM DATA POINTS
total_samples = 5
x_train, y_train = make_classification(n_samples=total_samples,n_features=2, n_redundant=0, n_informative=1,
                             n_clusters_per_class=1)
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

x_test, y_test = make_classification(n_samples=total_samples,n_features=2, n_redundant=0, n_informative=1,
                             n_clusters_per_class=1)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test)

In [4]:
model = Feedforward(2, 4)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
print(model)

Feedforward input_size  2  hidden_size  4
Feedforward(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [5]:
model.eval()
y_pred = model(x_test)
before_train = criterion(y_pred.squeeze(), y_test)
print('Test loss before training' , before_train.item())

#input      [5, 2]  our input shape
#hidden fc1 [5, 4]  (fc1): Linear(in_features=2, out_features=4, bias=True) 
#relu       [5, 4]  relu same size
#output fc2 [5, 1]  Linear(in_features=4, out_features=1, bias=True)
#sigmoid_output  [5, 1]  Sigmoid

input
[5, 2]
tensor([[-0.8300,  0.3065],
        [-1.0359, -0.9094],
        [-1.0609, -0.3498],
        [ 1.5576,  1.2509],
        [ 1.5136,  0.0725]])
hidden fc1
[5, 4]
tensor([[ 0.7310,  1.0147,  0.2710, -0.2432],
        [ 0.0648,  0.6299, -0.2630,  0.4065],
        [ 0.3659,  0.8893,  0.0429,  0.1143],
        [ 1.3517, -0.2168, -0.4325, -0.8751],
        [ 0.7132, -0.6967, -1.0281, -0.2543]], grad_fn=<AddmmBackward>)
relu
[5, 4]
tensor([[0.7310, 1.0147, 0.2710, 0.0000],
        [0.0648, 0.6299, 0.0000, 0.4065],
        [0.3659, 0.8893, 0.0429, 0.1143],
        [1.3517, 0.0000, 0.0000, 0.0000],
        [0.7132, 0.0000, 0.0000, 0.0000]], grad_fn=<ThresholdBackward0>)
output fc2
[5, 1]
tensor([[-0.0360],
        [-0.0506],
        [-0.0020],
        [-0.0686],
        [-0.1138]], grad_fn=<AddmmBackward>)
sigmoid_output
[5, 1]
tensor([[0.4910],
        [0.4874],
        [0.4995],
        [0.4829],
        [0.4716]], grad_fn=<SigmoidBackward>)
Test loss before training 0.703066051006

In [6]:
model.train()
epoch = 20
for epoch in range(epoch):    
    optimizer.zero_grad()    
    # Forward pass
    y_pred = model(x_train)    
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y_train)
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))    # Backward pass
    loss.backward()
    optimizer.step()

Epoch 0: train loss: 0.6781232953071594
Epoch 1: train loss: 0.6776665449142456
Epoch 2: train loss: 0.6772103309631348
Epoch 3: train loss: 0.6767548322677612
Epoch 4: train loss: 0.6762996912002563
Epoch 5: train loss: 0.6758453845977783
Epoch 6: train loss: 0.675391674041748
Epoch 7: train loss: 0.674938440322876
Epoch 8: train loss: 0.6744857430458069
Epoch 9: train loss: 0.6740336418151855
Epoch 10: train loss: 0.6735818982124329
Epoch 11: train loss: 0.6731308698654175
Epoch 12: train loss: 0.6726801991462708
Epoch 13: train loss: 0.672230064868927
Epoch 14: train loss: 0.6717803478240967
Epoch 15: train loss: 0.6713311672210693
Epoch 16: train loss: 0.6708823442459106
Epoch 17: train loss: 0.6704340577125549
Epoch 18: train loss: 0.6699860692024231
Epoch 19: train loss: 0.6695384979248047


In [7]:
model.eval()
y_pred = model(x_test)
after_train = criterion(y_pred.squeeze(), y_test) 
print('Test loss after Training' , after_train.item())

Test loss after Training 0.69452965259552


In [11]:
after_train

tensor(0.6945, grad_fn=<BinaryCrossEntropyBackward>)